In [1]:
# importing libraries
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image

In [2]:

mtcnn = MTCNN(image_size=240, margin=0, min_face_size=20) # initializing mtcnn for face detection
resnet = InceptionResnetV1(pretrained='vggface2').eval() # initializing resnet for face img to embeding conversion

dataset=datasets.ImageFolder('photos') # photos folder path 
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = [] # list of cropped faces from photos folder
name_list = [] # list of names corrospoing to cropped photos
embedding_list = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using resnet

for img, idx in loader:
    face, prob = mtcnn(img, return_prob=True) 
    if face is not None and prob>0.90: # if face detected and porbability > 90%
        emb = resnet(face.unsqueeze(0)) # passing cropped face into resnet model to get embedding matrix
        embedding_list.append(emb.detach()) # resulten embedding matrix is stored in a list
        name_list.append(idx_to_class[idx]) # names are stored in a list
        


In [3]:
data = [embedding_list, name_list]
torch.save(data, 'data.pt') # saving data.pt file

In [4]:
import cv2
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def draw_rect(img, faces, sf=20):
    for x, y, w, h in faces:
        x = x - sf
        y = y - 2*sf
        cv2.rectangle(img, (x,y), (x+w+2*sf, y+h+3*sf), (255, 0 , 0), 3)

        
        
       
cap = cv2.VideoCapture(0)



frameFrequency=25
total_frame = 0
id = 0

while cap.isOpened():
    ret, frame = cap.read(0)

    if ret is False:
        break

    faces = face_detector.detectMultiScale(frame, 1.1, 4)
    print(faces)

    draw_rect(frame, faces)
    
    total_frame += 1
    if total_frame%frameFrequency == 0:
        id += 1
        image_name = "user" + str(id) +'.jpg'
        cv2.imwrite(image_name, frame)
        print(image_name)
    # Display the output
    cv2.imshow('Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    if id >=5:        
        break

cap.release()
cv2.destroyAllWindows()



def face_match(img_path, data_path): # img_path= location of photo, data_path= location of data.pt 
    # getting embedding matrix of the given img
    img = Image.open(img_path)
    face, prob = mtcnn(img, return_prob=True) # returns cropped face and probability
    emb = resnet(face.unsqueeze(0)).detach() # detech is to make required gradient false
    
    saved_data = torch.load('data.pt') # loading data.pt file
    embedding_list = saved_data[0] # getting embedding data
    name_list = saved_data[1] # getting list of names
    dist_list = [] # list of matched distances, minimum distance is used to identify the person
    
    for idx, emb_db in enumerate(embedding_list):
        dist = torch.dist(emb, emb_db).item()
        dist_list.append(dist)
        
    idx_min = dist_list.index(min(dist_list))
    return (name_list[idx_min], min(dist_list))



z=0
total_images_to_be_clicked=5
for i in range(1,6):
    name = "user" +str(i) +".jpg"
# name="user3.jpg"
    result = face_match(name, 'data.pt')
    
    z = z+ result[1]
    
z=z/total_images_to_be_clicked
if(z>0.8):
    print("Image didn't match")
elif(z<=0.8):
    print("Image matched with existing user")

print('Face matched with: ',result[0], 'With distance: ',z)

[[251 248 194 194]]
[[233 218 232 232]]
[[234 207 233 233]]
[[236 195 232 232]]
[[238 185 234 234]]
[[241 176 245 245]]
[[247 166 254 254]]
[[256 159 258 258]]
[[261 147 263 263]]
[[255 114 283 283]]
[[252  90 278 278]]
[[248  79 275 275]]
[[237  69 272 272]]
[[227  83 266 266]]
[[229 142 258 258]]
[[232 167 253 253]]
[[231 172 250 250]]
[[233 177 243 243]]
[[238 184 244 244]]
[[248 195 236 236]]
[[249 198 237 237]]
[[257 201 226 226]]
[[260 201 223 223]]
[[264 202 214 214]]
[[262 198 218 218]]
user1.jpg
[[261 202 213 213]]
[[252 200 223 223]]
[[248 202 219 219]]
[[242 202 218 218]]
[[239 200 225 225]]
[[235 205 216 216]]
[[233 206 216 216]]
[[235 207 211 211]]
[[231 207 213 213]]
[[230 207 212 212]]
[[229 205 215 215]]
[[232 207 211 211]]
[[230 203 216 216]]
[[230 204 212 212]]
[[229 204 214 214]]
[[230 204 215 215]]
[[233 205 212 212]]
[[234 206 210 210]]
[[234 206 210 210]]
[[235 206 209 209]]
[[235 205 210 210]]
[[237 206 209 209]]
[[237 206 209 209]]
[[234 205 211 211]]
[[234 205 